In [118]:
import pandas as pd
import numpy as np
import biosppy.signals.ecg as ecg

from sklearn import preprocessing
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
def load_data():
    X_train = pd.read_csv("X_train.csv")
    X_test = pd.read_csv("X_test.csv")
    y_train = pd.read_csv("y_train.csv")

    #dropping id column
    X_train = X_train.drop('id', axis = 1)
    X_test = X_test.drop('id', axis = 1)
    y_train = y_train.drop('id', axis = 1)
    
    #reshuffling data
    X_train['y'] = y_train
    X_train = X_train.sample(frac=1).reset_index(drop=True)
    y_train = X_train['y']
    X_train = X_train.drop('y', axis = 1)
    
    return X_train, X_test, y_train

In [3]:
xtrain, xtest, ytrain = load_data()

Goal in code below

Repeat for all training examples:  
    First extract r peaks from signal with ssf_segmenter (or any other function that works)  
    Then use that together with the signal to get heartbeats  
    Heartbeat function outputs an n x m array where n are the amount of heartbeats and m (=180) are the features per     heartbeat  
    Compute mean, median and standard deviation over all the heartbeat features (=> 3*180 = 540 features) and insert     in dataframe  

In [107]:
# Create an empty data frame with 180 * 3 columns
training_set = pd.DataFrame(columns = np.arange(540))

for index, row in xtrain.iterrows():
    
    signal_array = row.values
    rpeaks = ecg.ssf_segmenter(signal_array, sampling_rate = 300)
    rpeaks = np.array(rpeaks).ravel()
    heartbeat, rpeaks = ecg.extract_heartbeats(signal = signal_array, 
                                          rpeaks = rpeaks, sampling_rate = 300)
    
    # Iterate through all the heartbeat features and compute mean, median and standard deviation
    means = np.array([])
    medians = np.array([])
    stds = np.array([])
    for feat in heartbeat.T:
        means = np.append(means, np.mean(feat))
        medians = np.append(medians, np.median(feat))
        stds = np.append(stds, np.std(feat))
    
    # Add as training example
    values = np.append(means, medians)
    values = np.append(values, stds)
    training_set.loc[index] = values
    
        

In [111]:
pd.set_option('display.max_columns', 600)
training_set.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539
0,-2.702811,-1.530120,-0.493976,1.004016,2.759036,4.405622,5.763052,6.899598,8.072289,9.024096,9.425703,9.353414,9.329317,9.353414,9.646586,10.425703,11.883534,13.799197,15.481928,17.016064,18.369478,19.626506,20.321285,20.843373,21.421687,21.738956,21.730924,21.313253,20.883534,20.574297,20.433735,20.714859,21.060241,21.578313,21.694779,21.265060,20.200803,19.052209,17.722892,16.530120,16.052209,16.192771,16.188755,16.405622,16.722892,16.987952,17.188755,17.277108,17.449799,18.152610,19.755020,21.995984,25.401606,30.220884,36.943775,46.224900,56.771084,67.497992,77.204819,82.987952,83.253012,76.771084,63.606426,46.377510,29.441767,16.333333,8.140562,3.036145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-25.0,-24.0,-23.0,-21.0,-23.0,-22.0,-23.0,-21.0,-20.0,-19.0,-17.0,-16.0,-15.0,-15.0,-14.0,-12.0,-10.0,-9.0,-8.0,-7.0,-7.0,-3.0,-3.0,-2.0,-2.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-3.0,-2.0,-2.0,-4.0,-6.0,-6.0,-4.0,-6.0,-6.0,-6.0,-7.0,-8.0,-4.0,-5.0,-3.0,-3.0,-2.0,0.0,2.0,6.0,11.0,19.0,31.0,41.0,49.0,52.0,50.0,48.0,44.0,38.0,31.0,15.0,-1.0,-12.0,-17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142.929027,143.746207,144.217098,1

In [123]:
training_set.dropna(axis = 1).shape

(5117, 180)

In [121]:
steps = [("scaler", preprocessing.StandardScaler()), ("classifier", SVC())]

pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
              "classifier__class_weight": ["balanced"]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1_micro')

grid.fit(training_set.dropna(axis = 1).values, ytrain.values.ravel())

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('classifier',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
         

In [122]:
grid.best_score_

0.5614617940199336